In [12]:
import os
import json
# !pip install pandas
import pandas as pd
import traceback

In [17]:
# !pip install langchain-openai
from langchain_openai import ChatOpenAI


In [24]:
# !pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
KEY=os.getenv("OPENAI_API_KEY")

In [27]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-4o-mini", temperature=0.7)

In [29]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000153251AAF60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000015325101070>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')

In [38]:
# !pip install langchain
# !pip install  langchain_community
# !pip install  PyPDF2

from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_community.callbacks import get_openai_callback
import PyPDF2

In [46]:
RESPONSE_JSON={
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "answer1",
            "b": "answer2",
            "c": "answer3",
            "d": "answer4"
        },
        "correct": "correcr answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "answer1",
            "b": "answer2",
            "c": "answer3",
            "d": "answer4"
        },
        "correct": "correcr answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "answer1",
            "b": "answer2",
            "c": "answer3",
            "d": "answer4"
        },
        "correct": "correcr answer"
    },
}

In [73]:
TEMPLATE=""""
Text: {text}
Your are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for 
{subject} students in {tone}.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONS_JSON below and use it as a guide.
Ensure to make {number} MCQs

{response_json}
"""

In [74]:
quiz_generator_promp =  PromptTemplate(
    input_variables=['tex', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [75]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generator_promp, output_key="quiz", verbose=True)

In [76]:
TEMPLATE2="""
You are an expert english grammerian and writer. Givern a Multiple Choice Quiz for {subject} students. \
you need to evaluate the complexity of the question and give a cimplete analysis of the quiz. Only use at max 50 word for the complexity.
if the quiz is not at per with the cognitive and analyitical abilities of the students, \
update the quiz questions which needs to be changed and the tone such that it perfectly fit the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [77]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [78]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True )

In [79]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [80]:
file_path=r"C:\Users\Saudinnov\Desktop\mcqgen\data.txt"

In [81]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [82]:
# print(TEXT)

In [83]:
#Serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "answer1", "b": "answer2", "c": "answer3", "d": "answer4"}, "correct": "correcr answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "answer1", "b": "answer2", "c": "answer3", "d": "answer4"}, "correct": "correcr answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "answer1", "b": "answer2", "c": "answer3", "d": "answer4"}, "correct": "correcr answer"}}'

In [84]:
NUMBER=5
SUBJECT="Generative AI"
TONE='simple'

In [85]:
#  how to setup Token Usage Tracking in Langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"
Text: Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed gener

In [86]:
print(f"total tokens:{cb.total_tokens}")
print(f"Prompt tokens:{cb.prompt_tokens}")
print(f"Completion tokens:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")

total tokens:6838
Prompt tokens:5871
Completion tokens:967
total cost:0.0014608499999999999


In [87]:
response

{'text': 'Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]\n\nGenerative AI has uses across a wide range of industries, including

In [88]:
quiz=response.get('quiz')

In [97]:
quiz=json.loads(quiz)

In [98]:
review=response.get('review')

**Complexity Analysis:**
The quiz questions are generally appropriate for students studying generative AI, focusing on basic definitions, historical developments, and ethical concerns. However, the questions could benefit from enhanced cognitive engagement and analytical depth. 

**Updated Quiz Questions:**

```json
{
  "1": {
    "mcq": "What is the core function of generative AI?",
    "options": {
      "a": "To create new content based on input data",
      "b": "To analyze and categorize existing data",
      "c": "To archive information for future reference",
      "d": "To protect data from unauthorized access"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "Identify the model type that is NOT typically associated with generative AI.",
    "options": {
      "a": "Text-to-Image models",
      "b": "Text-to-Video models",
      "c": "Text-to-Speech models",
      "d": "Text-to-Emotion models"
    },
}
```
**Rationale for Changes:**
1. **Clarity and Engagement:** Updated wording for clarity and engagement while maintaining essential content.
2. **Cognitive Depth:** Enhanced phrasing encourages critical thinking by inviting students to reflect on the implications of generative AI.
3. **Tone and Complexity:** Adjusted tone to be slightly more formal and analytical, aligning with students' cognitive abilities.

In [102]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct = value['correct']
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [107]:
quiz_table_data

[{'MCQ': 'What does generative AI primarily do?',
  'Choices': 'a: Generate data in response to prompts | b: Only process existing data | c: Store historical data | d: Secure data from cyber threats',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a type of generative AI model mentioned in the text?',
  'Choices': 'a: Text-to-Image models | b: Text-to-Video models | c: Text-to-Speech models | d: Text-to-Emotion models',
  'Correct': 'd'},
 {'MCQ': 'What significant advancement in AI occurred in 2017?',
  'Choices': 'a: Release of GPT-4 | b: Introduction of the Transformer network | c: Development of audio deepfakes | d: Creation of the first AI-generated video',
  'Correct': 'b'},
 {'MCQ': 'What is a major concern regarding the misuse of generative AI?',
  'Choices': 'a: Improvement of job efficiency | b: Creation of deepfakes and fake news | c: Enhancement of customer service | d: Development of new programming languages',
  'Correct': 'b'},
 {'MCQ': 'Which country is repo

In [111]:
# !pip install numpy --upgrade

In [114]:
import pandas as pd
quiz=pd.DataFrame(quiz_table_data)

In [117]:
quiz.to_csv("generativeai.csv", index=False)